<a href="https://colab.research.google.com/github/tanu011/Gesture-Controlled-Drone/blob/main/image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install SimpleITK vtk pyvista numpy


In [8]:
import SimpleITK as sitk
import numpy as np
import pyvista as pv
import os

In [14]:
def load_ct_images(directory):
    reader = sitk.ImageSeriesReader()
    series_ids = reader.GetGDCMSeriesIDs(directory)
    if not series_ids:
        raise ValueError(f"No DICOM series found in directory: {directory}")
    series_file_names = reader.GetGDCMSeriesFileNames(directory, series_ids[0])
    reader.SetFileNames(series_file_names)
    image = reader.Execute()
    return image


In [10]:
def preprocess_image(image):
    # window level filter to enhance bone contrast
    image = sitk.IntensityWindowing(image, windowMinimum=500, windowMaximum=3000)
    return image

In [11]:
def segment_skull(image):
    # Thresholding to isolate the bone
    bone_threshold = sitk.BinaryThreshold(image, lowerThreshold=500, upperThreshold=3000)

    # Connected component analysis to ensure connectivity
    connected_components = sitk.ConnectedComponent(bone_threshold)
    largest_component = sitk.RelabelComponent(connected_components)
    skull_segmentation = sitk.BinaryThreshold(largest_component, lowerThreshold=1, upperThreshold=1)

    return skull_segmentation

In [12]:
def export_stl(segmentation, filename):
    # Convert the SimpleITK image to a numpy array
    np_array = sitk.GetArrayFromImage(segmentation).astype(np.uint8)

    # Create a PyVista grid
    grid = pv.UniformGrid()
    grid.dimensions = np.array(segmentation.GetSize()) + 1
    grid.origin = segmentation.GetOrigin()
    grid.spacing = segmentation.GetSpacing()
    grid.cell_data["values"] = np_array.flatten(order="F")

    # Create a surface from the segmentation
    surface = grid.contour([0.5])

    # Save as STL
    surface.save(filename)


In [16]:
directory = "path/to/ct/images"

# Check if directory exists
if not os.path.exists(directory):
    raise ValueError(f"The specified directory does not exist: {directory}")


# Load, preprocess, segment, and export the skull
try:
    image = load_ct_images(directory)
    preprocessed_image = preprocess_image(image)
    skull_segmentation = segment_skull(preprocessed_image)
    export_stl(skull_segmentation, "skull.stl")
    print("Skull segmentation and export completed successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

ValueError: The specified directory does not exist: path/to/ct/images